In [1]:
using LinearAlgebra
using QuantumOptics
using DynamicPolynomials, MomentTools

# Quantum system

In [2]:
#= 
Quantum system is taken from 
 https://github.com/q-optimize/c3/blob/master/examples/two_qubits.ipynb

Drift Hamiltonian
=#

H0 = [
    0 0 0;
    0 3.21505101e+10 0;
    0 0 6.23173079e+10
];

H0 /= norm(H0, Inf)

# Control Hamiltonian
V = [
    0 1 0;
    1 0 1.41421356;
    0 1.41421356 0
]

V /= norm(V, Inf);

# Symbolic integration utilities (had to implement them by myself)

In [3]:
function ∫(p::AbstractPolynomial, x::AbstractVariable, x_lower, x_upper)
    
    # get the index of the variable of integration
    ind_x = indexin([x], variables(p))[1]
        
    if isnothing(ind_x)
        # integration valuable is not found among vars
        return p * (x_upper - x_lower)
    end
    
    # get the indefinite integral
    int_p = sum(
        coeff * term * x * 1 // (exponents(term)[ind_x] + 1)
        for (coeff, term) in zip(coefficients(p), terms(p));
        init = 0 * x
    )
        
    # get the definite integral
    subs(int_p, x=>x_upper) - subs(int_p, x=>x_lower)
end

function ∫(M::AbstractMatrix, x::AbstractVariable, x_lower, x_upper)
   map(z -> ∫(z, x, x_lower, x_upper), M) 
end

∫ (generic function with 2 methods)

In [18]:
@polyvar x[1:3]
@polyvar t[1:3]

# final time
const T = 1

function u(t, x)
    # the polynomial shape for control
    sum(x[n] * t^n for n = 1:length(x))
end


###############################################################################
# Get the target unitary

# randomly generate the coefficients of the polynomial control
exact_x = 0.1 * randn(length(x)) # normal random variable



b = NLevelBasis(size(H0)[1])

𝓗₀ = DenseOperator(b, b, H0)
𝓥 = DenseOperator(b, b, V)

function 𝓗(t, psi)
  return 𝓗₀ + 𝓥 * u(t, exact_x)
end

tout, 𝓤 = timeevolution.schroedinger_dynamic([0:T;], identityoperator(b,b), 𝓗)

# target unitray
U_target = Matrix(𝓤[2].data)

###############################################################################


function A(t)
    #=
    The generator of motion entering the Magnus expansion
    :param u: control
    :param t:  time variable
    :return: matrix
    =#
    (H0 + V * u(t, x)) / im
end

function commutator(a, b)
    a * b - b * a
end 

###############################################################################
# get the partial sum of the Magnus expansion
A₁ = A(t[1])
A₂ = A(t[2])
A₃ = A(t[3])

Ω = ∫(A₁, t[1], 0, T)

Ω += 1//2 * ∫(∫(
    commutator(A₁, A₂), 
    t[2], 0, t[1]), 
    t[1], 0, T
)

# Omega3 = Rational(1, 6) * integrate(integrate(integrate(
#    commutator(A1, commutator(A2, A3))
#    + commutator(commutator(A1, A2), A3),
#    (t3, 0, t2)), (t2, 0, t1)), (t1, 0, T)
# )

###############################################################################

# In the next section we evaluate the above approximation for \tanh(\Omega/2) and save it as approx_tanh.
# Note that we empirically found that it is good to keep first 3 terms in the Taylor expansion for tanh

Ω = convert(typeof(A₁), Ω)

Ω²¼ = Ω^2 / 4

approx_tanh = (Ω/2) * (I + Ω²¼ * (-1//3 * I + 2//15 * Ω²¼))


obj = I - U_target + (I + U_target) * approx_tanh


function real_poly(p::Polynomial)
    #=
    Real part of the polynomial
    =#
    sum(
        real(c) * m for (c, m) in zip(coefficients(p), monomials(p))
    )
end


obj = real_poly(tr(obj * obj'));

obj(exact_x)

2.001076738848816

In [77]:
using MosekTools
using JuMP

optimizer = optimizer_with_attributes(Mosek.Optimizer, "QUIET" => true)
v, M = minimize(obj, [], [], variables(obj), 10, optimizer)

(1.9737825306463386, A JuMP Model
Minimization problem with:
Variables: 1771
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 1 constraint
`Vector{AffExpr}`-in-`MathOptInterface.PositiveSemidefiniteConeTriangle`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: basis, degree, dual, index, moments, monomials, nu, type, variables, y)

In [78]:
r = get_minimizers(M)

3×149 Matrix{Float64}:
 -3.15998  -2.88097   -2.73645  …  2.67997  2.67698  2.8636    3.17415
 -1.80997  -0.372281  -1.72714     1.21251  1.82328  0.400108  1.821
 -3.32297  -2.91667   -2.36483     2.48676  2.40568  2.94243   3.31321

In [79]:
min([obj(r[:,indx]) for indx=1:size(r)[2]]...) - v

4.243410280246884e-5

In [82]:
[obj(r[:,indx]) for indx=1:size(r)[2]]

149-element Vector{Float64}:
 34.13692646484685
 10.213253068498283
 14.376293809971443
 11.119771483046053
  6.804446247298103
  8.512726892714545
  9.3948639172089
  6.650925311929859
  6.056264254107954
  6.685887711118211
  6.337091861811945
  6.150758305215899
  5.020992443224238
  ⋮
  6.622361553927516
  6.659095752946971
  7.123779616235665
  6.391246694649717
  7.0152452756092805
 10.041991907598169
  9.208540384284905
  7.301889009832049
 12.09450234313131
 15.931855781923154
 11.058506691557024
 38.52823051838681

In [55]:
r

3×30 Matrix{Float64}:
 -7.75431  -2.34921  -4.15503  -1.3932   …   9.00788   7.53668   5.95044
  7.23509   4.54047   3.5735    3.79738      6.51475  -3.38563  -7.56709
 -3.92033  -5.31136  -1.04521  -2.9399      -3.87693  -0.36658   6.62907